In [8]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time


In [9]:
static_path='/Users/saml16/projects/Elections_info/static/lok_sabha_2024'
constituency_list_file_path=static_path+'/constituency_list.csv'
phase='/phase_2'
constituency_links_file_path=static_path+phase+'/constituency_links_list.csv'
constituency_html_file_path=static_path+phase+'/constituency.html'
constituency_folder_path=static_path+phase+'/constituency_html_dump/'

fetched_constituencies_ids_file_path='/Users/saml16/projects/Elections_info/static/lok_sabha_2024/constituency_list.csv'

"""/Users/saml16/projects/Elections_info/static/lok_sabha_2024/phase_2/constituency_html_dump
/Users/saml16/projects/Elections_info/static/lok_sabha_2024/phase_2/constituency_candidates_data
'/Users/saml16/projects/Elections_info/static/lok_sabha_2024/phase_1/phase1_constituency_list_links_.csv"""

"/Users/saml16/projects/Elections_info/static/lok_sabha_2024/phase_2/constituency_html_dump\n/Users/saml16/projects/Elections_info/static/lok_sabha_2024/phase_2/constituency_candidates_data\n'/Users/saml16/projects/Elections_info/static/lok_sabha_2024/phase_1/phase1_constituency_list_links_.csv"

In [5]:
# fetch the constituency url html
def fetch_constituency_links():
    
    url = 'https://www.myneta.info/LokSabha2024/'
    # Part of the href to search for in the links
    search_query = 'action=show_candidate'
    
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all <a> tags that contain the search query in their href attribute
    a_tags = soup.find_all('a', href=lambda href: href and search_query in href)
    
    data = []
    
    for a in a_tags:
        link_={}
        # Get the link text and href
        link_['constituency'] = a.text.strip()
        link_['constituency_link'] = a.get('href')

        # Navigate to the grandparent and find the button
        grandparent = a.find_parent().find_parent()
        button = grandparent.find('button')
        link_['state'] = button.text.strip() if button else "None"
        link_['constituency_id'] = link_['constituency_link'].split('=')[2] if link_['constituency_link'] else 0

        data.append(link_)
    return pd.DataFrame(data)


check=fetch_constituency_links()

check



,constituency,constituency_link,state,constituency_id
0,ANDAMAN AND NICOBAR ISLANDS,index.php?action=show_candidates&constituency_...,ANDAMAN AND NICOBAR ISLANDS,579
1,ARUNACHAL EAST,index.php?action=show_candidates&constituency_...,ARUNACHAL PRADESH,29
2,ARUNACHAL WEST,index.php?action=show_candidates&constituency_...,ARUNACHAL PRADESH,28
3,DARRANG-UDALGURI,index.php?action=show_candidates&constituency_...,ASSAM,35
4,DIBRUGARH,index.php?action=show_candidates&constituency_...,ASSAM,33
...,...,...,...,...
184,BALURGHAT,index.php?action=show_candidates&constituency_...,WEST BENGAL,539
185,COOCH BEHAR (SC),index.php?action=show_candidates&constituency_...,WEST BENGAL,534
186,DARJEELING,index.php?action=show_candidates&constituency_...,WEST BENGAL,537
187,JALPAIGURI (SC),index.php?action=show_candidates&constituency_...,WEST BENGAL,536


In [19]:


def fetch_constituency_html(constituency_ids,output_path):
 
    # remove existing ids. 
    # fetch html of the constituencies.


    fetched_constituencies= pd.read_csv(fetched_constituencies_ids_file_path)
    fetched_constituencies_ids=set(fetched_constituencies['constituency_ids'])
    
    for constituency_id in constituency_ids:
        if int(constituency_id) not in fetched_constituencies_ids and int(constituency_id)!=0:
            r = requests.get("https://www.myneta.info/LokSabha2024/index.php?action=show_candidates&constituency_id="+str(constituency_id))
            #r.content
            soup = BeautifulSoup(r.content, 'html.parser')
            name = output_path+str(constituency_id)+".html"
            with open( name, "w", encoding='utf-8') as file :
                file.write(str(soup.prettify()))
            
            fetched_constituencies_ids.add(constituency_id)
            
            
            time.sleep(2)
            
            print("Done for id: "+ str(constituency_id))
        else:
            print("Skipping for fetched id: "+ str(constituency_id))
        
        pd.DataFrame(list(fetched_constituencies_ids),columns=['constituency_ids']).to_csv(fetched_constituencies_ids_file_path,index=False)
    

fetch_constituency_html(check['constituency_id'],constituency_folder_path)         
            


Skipping for fetched id: 579
Skipping for fetched id: 29
Skipping for fetched id: 28
Skipping for fetched id: 35
Skipping for fetched id: 33
Skipping for fetched id: 36
Skipping for fetched id: 34
Skipping for fetched id: 37
Skipping for fetched id: 30
Skipping for fetched id: 32
Skipping for fetched id: 39
Skipping for fetched id: 38
Skipping for fetched id: 31
Skipping for fetched id: 45
Skipping for fetched id: 53
Skipping for fetched id: 52
Skipping for fetched id: 46
Skipping for fetched id: 48
Skipping for fetched id: 50
Skipping for fetched id: 49
Skipping for fetched id: 47
Done for id: 51
Skipping for fetched id: 89
Done for id: 92
Done for id: 91
Done for id: 90
Done for id: 156
Skipping for fetched id: 155
Done for id: 185
Done for id: 184
Done for id: 183
Done for id: 186
Done for id: 182
Done for id: 187
Done for id: 178
Done for id: 177
Done for id: 176
Done for id: 188
Done for id: 180
Done for id: 181
Done for id: 179
Done for id: 175
Done for id: 220
Done for id: 213
D

In [20]:
# Save the constituency list for this pahse 
check.to_csv(constituency_links_file_path,index=False)

In [ ]:

# fetch users data
# fetch users images
# sort the data into required formats 


In [22]:
!pip install requests_html

  Obtaining dependency information for requests_html from https://files.pythonhosted.org/packages/24/bc/a4380f09bab3a776182578ce6b2771e57259d0d4dbce178205779abdc347/requests_html-0.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyquery from https://files.pythonhosted.org/packages/36/b7/f7ccf9e52e2817e1265d3719c600fa4ef33c07de4d5ef0ced3f43ab1cef2/pyquery-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for fake-useragent from https://files.pythonhosted.org/packages/e4/99/60d8cf1b26938c2e0a57e232f7f15641dfcd6f8deda454d73e4145910ff6/fake_useragent-1.5.1-py3-none-any.whl.metadata
  Obtaining dependency information for parse from https://files.pythonhosted.org/packages/ce/f0/30fe1494f1910ad3ea40639b13ac48cdb16a8600e8861cbfc2c560661ddf/parse-1.20.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pyppeteer>=0.0.14 from https://files.pythonhosted.org/packages/3d/ee/fb2757a38025421fd3844a0ed0a230b78c9c04a66355024436cf3005a70c/pyppete

In [23]:
from requests_html import HTMLSession

# URL of the webpage containing the JavaScript
url = 'https://www.myneta.info/LokSabha2024/index.php?action=show_candidates&constituency_id=205'

# Create an HTMLSession object
session = HTMLSession()

# Use the HTMLSession object to get the content
response = session.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Render JavaScript by calling render() method
    response.html.render()

    # Save the fully rendered HTML content to a file
    with open('output.html', 'w', encoding='utf-8') as f:
        f.write(response.html.html)
    print("Content saved successfully!")
else:
    print("Failed to fetch content. Status code:", response.status_code)

RuntimeError: Cannot use HTMLSession within an existing event loop. Use AsyncHTMLSession instead.

In [24]:
from requests_html import AsyncHTMLSession
import asyncio

async def fetch_and_save_content(url):
    # Create an AsyncHTMLSession object
    session = AsyncHTMLSession()

    # Use the AsyncHTMLSession object to get the content
    response = await session.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Render JavaScript by calling render() method
        await response.html.arender()

        # Save the fully rendered HTML content to a file
        with open('output.html', 'w', encoding='utf-8') as f:
            f.write(response.html.html)
        print("Content saved successfully!")
    else:
        print("Failed to fetch content. Status code:", response.status_code)

    # Close the session
    await session.close()

# URL of the webpage containing the JavaScript
#url = 'YOUR_URL_HERE'

# Run the asynchronous function in the event loop
asyncio.run(fetch_and_save_content(url))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [27]:
from requests_html import AsyncHTMLSession
import asyncio

async def fetch_and_save_content(url):
    # Create an AsyncHTMLSession object
    session = AsyncHTMLSession()

    # Use the AsyncHTMLSession object to get the content
    response = await session.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Render JavaScript by calling render() method
        await response.html.arender()

        # Save the fully rendered HTML content to a file
        with open('output.html', 'w', encoding='utf-8') as f:
            f.write(response.html.html)
        print("Content saved successfully!")
    else:
        print("Failed to fetch content. Status code:", response.status_code)

    # Close the session
    await session.close()

# URL of the webpage containing the JavaScript
url = 'https://www.myneta.info/LokSabha2024/index.php?action=show_candidates&constituency_id=205'

# Create an event loop
loop = asyncio.get_event_loop()

# Run the asynchronous function in the event loop
loop.run_until_complete(fetch_and_save_content(url))

# Close the event loop
loop.close()


RuntimeError: This event loop is already running

# Rough

In [10]:
import os
import csv

# Get all files in the current directory
files = os.listdir('/Users/saml16/projects/Elections_info/static/lok_sabha_2024/phase_1/constituency_html_dump/')

# Filter out the files that end with .html and remove the .html extension
html_files = [f[:-5] for f in files if f.endswith('.html')]

# Write the file names to a new file
with open('/Users/saml16/projects/Elections_info/static/lok_sabha_2024/constituency_list.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["constituency_ids"])
    for name in html_files:
        writer.writerow([name])

In [6]:
check = pd.read_csv(constituency_links_file_path)


In [7]:
check

,constituency,constituency_link,state,constituency_id
0,ANDAMAN AND NICOBAR ISLANDS,index.php?action=show_candidates&constituency_...,ANDAMAN AND NICOBAR ISLANDS,579
1,ARUNACHAL EAST,index.php?action=show_candidates&constituency_...,ARUNACHAL PRADESH,29
2,ARUNACHAL WEST,index.php?action=show_candidates&constituency_...,ARUNACHAL PRADESH,28
3,DARRANG-UDALGURI,index.php?action=show_candidates&constituency_...,ASSAM,35
4,DIBRUGARH,index.php?action=show_candidates&constituency_...,ASSAM,33
...,...,...,...,...
184,BALURGHAT,index.php?action=show_candidates&constituency_...,WEST BENGAL,539
185,COOCH BEHAR (SC),index.php?action=show_candidates&constituency_...,WEST BENGAL,534
186,DARJEELING,index.php?action=show_candidates&constituency_...,WEST BENGAL,537
187,JALPAIGURI (SC),index.php?action=show_candidates&constituency_...,WEST BENGAL,536


In [11]:

!pip install selenium

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/04/4d/a6e8afd65b87372e275eb612d564ec68f79195e9b7e27004a3b2cce69686/selenium-4.20.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/17/c9/f86f89f14d52f9f2f652ce24cb2f60141a51d087db1563f3fba94ba07346/trio-0.25.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing_extensions>=4.9.0 from https://files.pythonhosted.org/packages/01/f3/936e209267d6ef7510322191003885de524fc48d1b43269810cd589ceaf5/typing_extensions-4.11.0-py3-none-any.whl.metadata
  Obtaining dependency information for attrs>=23.2.0 from https://files.pythonhosted.org/packages/e0/44/827b2a91a5816512fcaf3cc4ebc465ccd5d598c45cefa670

In [13]:
from selenium import webdriver

# Set up the webdriver
driver = webdriver.Firefox()  # or webdriver.Chrome(), depending on your browser

# Open the HTML file
driver.get("https://www.myneta.info/LokSabha2024/index.php?action=show_candidates&constituency_id=186")

# Save the page source
page_source = driver.page_source

# Write the page source to a new HTML file
with open("output.html", "w") as f:
    f.write(page_source)

# Close the driver
driver.quit()